In [1]:
import pandas as pd
import numpy as np 
import seaborn as sns
import matplotlib.pyplot as plt
import string
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
import spacy

In [2]:
df=pd.read_csv('Language Detection.csv')

In [3]:
df.head()

,Text,Language
0,"Nature, in the broadest sense, is the natural...",English
1,"""Nature"" can refer to the phenomena of the phy...",English
2,"The study of nature is a large, if not the onl...",English
3,"Although humans are part of nature, human acti...",English
4,[1] The word nature is borrowed from the Old F...,English


In [4]:
df['Language'][7002]

'Danish'

In [5]:
def text_transform(text):
    for word in string.punctuation:
        text.replace(word,"")
    text=text.lower()
    return(text)

In [6]:
df["Text"]=df["Text"].apply(text_transform)

In [7]:
X=df.iloc[:,0]
y=df.iloc[:,1]

In [8]:
from sklearn.model_selection import train_test_split

In [9]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)

In [10]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [11]:
vec=TfidfVectorizer(ngram_range=(1,2),analyzer='char')

In [12]:
from sklearn import pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import accuracy_score

In [13]:
para={"criterion":["gini", "entropy", "log_loss"],
      'splitter':["best", "random"],
      'max_depth':[None,1,2,3,4,5,6,7,8,9,10],
      'min_samples_split':[1,2,3,4,5,6,7,8,9,10],
     }
      
      

In [14]:
rf=RandomForestClassifier()
dt=DecisionTreeClassifier()
svc=SVC()
xgb=XGBClassifier(objective='multi:softprob')
randomcv=RandomizedSearchCV(estimator=dt,param_distributions=para,n_jobs=-1,cv=3,scoring='accuracy')

In [15]:
model_pip1=pipeline.Pipeline([('vectorization',vec),('m1',rf)])

In [16]:
model_pip1.fit(X_train,y_train)

Pipeline(steps=[('vectorization',
                 TfidfVectorizer(analyzer='char', ngram_range=(1, 2))),
                ('m1', RandomForestClassifier())])

In [17]:
y_pred1=model_pip1.predict(X_test)

In [18]:
print("accuracy score for model1:-",accuracy_score(y_test,y_pred1))

accuracy score for model1:- 0.9700193423597679


**Model2:-**

In [19]:
X_train1=vec.fit_transform(X_train).toarray()
X_test1=vec.transform(X_test).toarray()

In [20]:
from sklearn.preprocessing import LabelEncoder
label=LabelEncoder()

In [21]:
y_train1=label.fit_transform(y_train)
y_test1=label.transform(y_test)

In [22]:
randomcv.fit(X_train1,y_train1)

C:\Users\win10\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\model_selection\_validation.py:547: FitFailedWarning: 
3 fits failed out of a total of 30.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
3 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\win10\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\model_selection\_validation.py", line 895, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\win10\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py", line 1467, in wrapper
    estimator._validate_params()
  File "C:\Users\win10\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py",

RandomizedSearchCV(cv=3, estimator=DecisionTreeClassifier(), n_jobs=-1,
                   param_distributions={'criterion': ['gini', 'entropy',
                                                      'log_loss'],
                                        'max_depth': [None, 1, 2, 3, 4, 5, 6, 7,
                                                      8, 9, 10],
                                        'min_samples_split': [1, 2, 3, 4, 5, 6,
                                                              7, 8, 9, 10],
                                        'splitter': ['best', 'random']},
                   scoring='accuracy')

In [23]:
randomcv.best_params_

{'splitter': 'random',
 'min_samples_split': 10,
 'max_depth': None,
 'criterion': 'entropy'}

In [24]:
y_pred2=randomcv.predict(X_test1)

In [25]:
print("accuracy score for model2:-",accuracy_score(y_test1,y_pred2))

accuracy score for model2:- 0.8573500967117988


**model3:-**

In [26]:
xgb.fit(X_train1,y_train1)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, objective='multi:softprob', ...)

In [27]:
y_pred3=xgb.predict(X_test1)

In [28]:
print("accuracy score for model3:-",accuracy_score(y_test1,y_pred3))

accuracy score for model3:- 0.9714700193423598


In [29]:
xgb.score(X_test1,y_test1)

0.9714700193423598

**model4:-**

In [31]:
svc.fit(X_train1,y_train1)

SVC()

In [32]:
y_pred4=svc.predict(X_test1)

In [33]:
print("accuracy score for model3:-",accuracy_score(y_test1,y_pred4))

accuracy score for model3:- 0.9835589941972921


In [2]:
d={"Model":["RandomForestClassifier","DecisionTreeClassifier","SVC","XGBClassifier"],
  "accuracy":[0.9700193423597679,0.8573500967117988,0.9835589941972921,0.9714700193423598]}

In [3]:
Data=pd.DataFrame(d)

In [4]:
Data

,Model,accuracy
0,RandomForestClassifier,0.970019
1,DecisionTreeClassifier,0.857350
2,SVC,0.983559
3,XGBClassifier,0.971470
